In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn import preprocessing
import xgboost as xgb
from sklearn.decomposition import PCA
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

In [ ]:
train_df = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
test_df = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
macro_df = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])
train_df = pd.merge(train_df, macro_df, how='left', on='timestamp').convert_objects(convert_numeric=True)
test_df = pd.merge(test_df, macro_df, how='left', on='timestamp').convert_objects(convert_numeric=True)
print(train_df.shape, test_df.shape)
id_test = test_df.id

In [ ]:
# we will add here some data cleaning

# Go to logarithms for target varialbe:
train_df['price_doc'] = np.log(1 + train_df['price_doc'])   

vars_to_logs = ['area_m']
for var in vars_to_logs:
    train_df[var] = np.log(1 + train_df[var])
    test_df[var] = np.log(1 + test_df[var])

columns = list(set(list(train_df.columns.values) + list(test_df.columns.values))) 
trash = ['child_on_acc_pre_school', 'old_education_build_share', 'modern_education_share']

columns_class = [f for f in columns if train_df[f].dtype=='object'] + [
        'ID_railroad_station_walk', 'ID_railroad_station_avto', 'ID_big_road1', 
        'ID_big_road2', 'ID_bus_terminal', 'ID_metro']

for f in columns_class:
    train_df[f] = train_df[f].fillna(-1)
    test_df[f] = test_df[f].fillna(-1)

# Computing average / mean / std per class
for f in columns_class:
    if f != 'price_doc' and not f in trash:
        print (f)
        # Median
        a = dict(train_df.groupby(f).median()['price_doc'])
        train_df[f + '_' + 'median'] = train_df[f].map(a).fillna(0)
        test_df[f + '_' + 'median'] = test_df[f].map(a).fillna(0)
        
        # Mean
        a = dict(train_df.groupby(f).mean()['price_doc'])
        train_df[f + '_' + 'mean'] = train_df[f].map(a).fillna(0)
        test_df[f + '_' + 'mean'] = test_df[f].map(a).fillna(0)
        
        # std
        a = dict(train_df.groupby(f).std()['price_doc'])
        train_df[f + '_' + 'std'] = train_df[f].map(a).fillna(0)
        test_df[f + '_' + 'std'] = test_df[f].map(a).fillna(0)


def convert_categoricals_to_dummies(train_df):
    for f in columns:
        if f != 'price_doc' and train_df[f].dtype=='object' and not f in trash:  
            print (f)
            # Converting categorical data to dummy variables
            dummies = pd.get_dummies(train_df[f]).rename(columns=lambda x: f + "_" + str(x))
            train_df = pd.concat([train_df, dummies], axis=1).drop(f, axis=1)
    return train_df

train_df = convert_categoricals_to_dummies(train_df)
test_df = convert_categoricals_to_dummies(test_df)

# This one is not present in test dataset, so we delete it
train_df = train_df.drop('sub_area_Poselenie Klenovskoe', 1)

#dealing with missing data
#missing data
total_train = train_df.isnull().sum().sort_values(ascending=False)
total_test = test_df.isnull().sum().sort_values(ascending=False)

train_df = train_df.drop((total_train[total_train > 0]).index,1, errors="ignore")
train_df = train_df.drop((total_test[total_test > 0]).index,1, errors="ignore")
test_df = test_df.drop((total_train[total_train > 0]).index,1, errors="ignore")
test_df = test_df.drop((total_test[total_test > 0]).index,1, errors="ignore")

    
# Deleting outliers...
train_df = train_df.drop(train_df[train_df['full_sq'] > 2000].index)

        
print ("Just to override previous one")

In [ ]:
features = list(train_df.columns)
features_remove = ['id', 'price_doc', 'timestamp']
features_m = [x for x in features if x not in features_remove and x not in trash]

#print (train_df.dtypes[:])
# Get only numerical variables
features_numbers = [features_m[i] for i in range(len(features_m)) if train_df.dtypes[i] == "int64" or train_df.dtypes[i] == "float64"]
print (features_numbers)

In [ ]:
#train_norm_df = (train_df[features_numbers] - train_df[features_numbers].mean()) / (train_df[features_numbers].max() - train_df[features_numbers].min())
#test_norm_df = (test_df[features_numbers] - train_df[features_numbers].mean()) / (train_df[features_numbers].max() - train_df[features_numbers].min())

#train_norm_df = preprocessing.normalize(train_df[features_numbers])

train_norm_df = (train_df[features_numbers] - train_df[features_numbers].mean()) / train_df[features_numbers].std()
test_norm_df = (test_df[features_numbers] - train_df[features_numbers].mean()) / train_df[features_numbers].std()

#train_norm_df = train_df[features_numbers]
#test_norm_df = test_df[features_numbers]

In [ ]:
pca = PCA(n_components=200)
pca.fit(train_norm_df)

train_pca_df = pca.transform(train_norm_df)
test_pca_df = pca.transform(test_norm_df)
print(pca.explained_variance_ratio_) 

In [ ]:
pca = PCA(n_components=200)
pca.fit(train_norm_df)

train_pca_df = pca.transform(train_norm_df)
test_pca_df = pca.transform(test_norm_df)
print(pca.explained_variance_ratio_) 

In [ ]:
xgb_params = {
    'eta': 0.025,
    'max_depth': 15,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1,
    'alpha' : 0.5,
    'lambda':0.0
}

dtrain = xgb.DMatrix(train_pca_df.as_matrix(), train_df['price_doc'].as_matrix(), feature_names=features_numbers)
dtest = xgb.DMatrix(test_pca_df.as_matrix(), feature_names=features_numbers)

In [ ]:
cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=10000, early_stopping_rounds=200,
    verbose_eval=50, show_stdv=False)
cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()

In [ ]:
num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)


fig, ax = plt.subplots(1, 1, figsize=(8, 13))
xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

In [ ]:
y_predict = np.exp(model.predict(dtest)) - 1
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.head()

In [ ]:
output.to_csv('xgbSub.csv', index=False)